# Student Database (MongoDB)


In [66]:
!pip install dnspython
!-m pip install pymongo[srv]

You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.
'-m' is not recognized as an internal or external command,
operable program or batch file.


In [64]:
import pymongo
import json
from pymongo import MongoClient, InsertOne

First create a database and then load the student.json dataset and Insert the students record into the collection

In [5]:
client = pymongo.MongoClient("mongodb://127.0.0.1:27017")
db = client.StudentDatabase
records = db.Studentslist
requesting = []

with open(r"C:/Users/Ram/students.json") as f:
  for jsonObj in f:
    myDict = json.loads(jsonObj)
    requesting.append(InsertOne(myDict))

result = records.bulk_write(requesting)

In [6]:
records.count_documents({})

200

In [35]:
students = records.find()
print(list(students))

[{'_id': 0, 'name': 'aimee Zank', 'scores': [{'score': 1.463179736705023, 'type': 'exam'}, {'score': 11.78273309957772, 'type': 'quiz'}, {'score': 35.8740349954354, 'type': 'homework'}]}, {'_id': 1, 'name': 'Aurelia Menendez', 'scores': [{'score': 60.06045071030959, 'type': 'exam'}, {'score': 52.79790691903873, 'type': 'quiz'}, {'score': 71.76133439165544, 'type': 'homework'}]}, {'_id': 2, 'name': 'Corliss Zuk', 'scores': [{'score': 67.03077096065002, 'type': 'exam'}, {'score': 6.301851677835235, 'type': 'quiz'}, {'score': 66.28344683278382, 'type': 'homework'}]}, {'_id': 3, 'name': 'Bao Ziglar', 'scores': [{'score': 71.64343899778332, 'type': 'exam'}, {'score': 24.80221293650313, 'type': 'quiz'}, {'score': 42.26147058804812, 'type': 'homework'}]}, {'_id': 4, 'name': 'Zachary Langlais', 'scores': [{'score': 78.68385091304332, 'type': 'exam'}, {'score': 90.2963101368042, 'type': 'quiz'}, {'score': 34.41620148042529, 'type': 'homework'}]}, {'_id': 5, 'name': 'Wilburn Spiess', 'scores': [

1) Find the student name who scored maximum scores in all (exam, quiz and homework)?

In [37]:
students1 = records.aggregate([
  { "$unwind": "$scores" },
  { "$group": {
      "_id": "$scores.type",
      "nameScores": {
        "$push": { "name": "$name", "score": "$scores.score" }
      },
      "max": { "$max": "$scores.score" }
    }
  },
  { "$set": {
      "max_score": {
        "$first": {
          "$filter": {
            "input": "$nameScores",
            "cond": { "$eq": [ "$$this.score", "$max" ] }
          }
        }
      }
    }
  },
  { "$project": {
      "_id": 0,
      "type": "$_id",
      "max_score": 1
    }
  }
])

In [38]:
print(list(students1))

[{'max_score': {'name': 'Whitley Fears', 'score': 99.77237745070993}, 'type': 'homework'}, {'max_score': {'name': 'Margart Vitello', 'score': 99.33685767140612}, 'type': 'exam'}, {'max_score': {'name': 'Cody Strouth', 'score': 99.80348240553108}, 'type': 'quiz'}]


2) Find students who scored below average in the exam and pass mark is 40%?


In [39]:
students2 = records.aggregate([
  { "$unwind": "$scores" },
  { "$group": {
      "_id": "$name",
      "nameScores": {
        "$push": { "name": "$name", "score": "$scores.score" }
      },
      "avgscore": { "$avg": "$scores.score" }
    }
  },
  { "$set": {
      "below_avg_score": {
        "$first": {
          "$filter": {
            "input": "$nameScores",
            "cond": { "$lt": [ "$$this.score", "$avgscore" ] }
          }
        }
      }
    }
  },
  { "$project": {
      "_id": 0,
      "below_avg_score": 1
    }
  }
])

In [40]:
print(list(students2))

[{'below_avg_score': {'name': 'Nobuko Linzey', 'score': 47.44831568815929}}, {'below_avg_score': {'name': 'Harriett Velarde', 'score': 33.7733570443736}}, {'below_avg_score': {'name': 'Gwen Honig', 'score': 76.66460454219344}}, {'below_avg_score': {'name': 'Synthia Labelle', 'score': 27.22049103148209}}, {'below_avg_score': {'name': 'Edgar Sarkis', 'score': 8.606983261043888}}, {'below_avg_score': {'name': 'Margart Vitello', 'score': 10.23046475899236}}, {'below_avg_score': {'name': 'Kaila Deibler', 'score': 15.89771199662455}}, {'below_avg_score': {'name': 'Tambra Mercure', 'score': 3.311794422000724}}, {'below_avg_score': {'name': 'Cassi Heal', 'score': 0.006247360551892012}}, {'below_avg_score': {'name': 'Tawana Oberg', 'score': 80.59006098671075}}, {'below_avg_score': {'name': 'Merissa Mann', 'score': 52.56522605123723}}, {'below_avg_score': {'name': 'Vina Matsunaga', 'score': 34.63479282877322}}, {'below_avg_score': {'name': 'Rudolph Domingo', 'score': 38.5413647805495}}, {'below_

3) Find students who scored below pass mark and assigned them as fail, and above pass mark as pass in all the categories

In [42]:
students3 = records.aggregate([
      { "$unwind": "$scores" },
      {"$project": {"_id":0,"name" :1, "scores.type":1, "scores.score":1, 
                    "result":{ 
                        "$cond": { "if" : { "$gte":["$scores.score",40]}, "then":"pass", "else":"fail"} } }}])

In [43]:
print(list(students3))

[{'name': 'aimee Zank', 'scores': {'score': 1.463179736705023, 'type': 'exam'}, 'result': 'fail'}, {'name': 'aimee Zank', 'scores': {'score': 11.78273309957772, 'type': 'quiz'}, 'result': 'fail'}, {'name': 'aimee Zank', 'scores': {'score': 35.8740349954354, 'type': 'homework'}, 'result': 'fail'}, {'name': 'Aurelia Menendez', 'scores': {'score': 60.06045071030959, 'type': 'exam'}, 'result': 'pass'}, {'name': 'Aurelia Menendez', 'scores': {'score': 52.79790691903873, 'type': 'quiz'}, 'result': 'pass'}, {'name': 'Aurelia Menendez', 'scores': {'score': 71.76133439165544, 'type': 'homework'}, 'result': 'pass'}, {'name': 'Corliss Zuk', 'scores': {'score': 67.03077096065002, 'type': 'exam'}, 'result': 'pass'}, {'name': 'Corliss Zuk', 'scores': {'score': 6.301851677835235, 'type': 'quiz'}, 'result': 'fail'}, {'name': 'Corliss Zuk', 'scores': {'score': 66.28344683278382, 'type': 'homework'}, 'result': 'pass'}, {'name': 'Bao Ziglar', 'scores': {'score': 71.64343899778332, 'type': 'exam'}, 'resul

4) Find the total and average of the exam, quiz and homework and store them in a separate collection.

In [44]:
students4 = records.aggregate(
[

 { "$unwind" : "$scores"},
  { "$group":
     {
       "_id": "$scores.type",
       "total" : { "$sum": "$scores.score" },
       "average":{ "$avg": "$scores.score"}
     }
 },
  {"$out": "studentstotal"}
 ])

In [46]:
students4 = db.studentstotal.find()
print(list(students4))

[{'_id': 'quiz', 'total': 9799.344638860508, 'average': 48.99672319430254}, {'_id': 'homework', 'total': 13563.739241322297, 'average': 67.81869620661149}, {'_id': 'exam', 'total': 9734.734151900351, 'average': 48.67367075950175}]


5)Create a new collection which consists of students who scored below average and above 40% in all the categories.

In [70]:
students5 =collection.aggregate([{"$unwind":"$scores"},
                       {"$group":{"_id":"$scores.type","Avg":{"$avg":"$scores.score"}}}])

for i in students5:
    print(i)

average=collection.aggregate([{"$match":{"scores.type":{"$in":["exam","quiz","homework"]},
                        "scores.0.score":{"$gt":40,"$lt":48.673},"scores.1.score":{"$gt":40,"$lt":48.996},
                                   "scores.2.score":{"$gt":40,"$lt":67.818}}},
                        {"$group":{"_id":"$_id","Name":{"$push":"$name"},"score":{"$push":"$scores.score"},
                                                                        "Type":{"$push":"$scores.type"}}},
                           {"$sort":{"_id":1}}])

for i in average:
    print(i)
else:
    print("No below average and above 40%")

{'_id': 'quiz', 'Avg': 48.99672319430254}
{'_id': 'homework', 'Avg': 67.81869620661149}
{'_id': 'exam', 'Avg': 48.67367075950175}
No below average and above 40%


6)Create a new collection which consists of students who scored below the fail mark in all the categorie

In [16]:
students6 = records.aggregate([
      { "$unwind": "$scores" },
      { "$match": { "scores.score": { "$lt": 40} }
       },
       { "$group": {"_id": "$_id", "fail": { "$push": { "name": "$name", "scores": "$scores"}}
       }
       },
       {"$out": "studentsfailed"}
      ])

In [48]:
students6 = db.studentsfailed.find()
print(list(students6))

[{'_id': 30, 'fail': [{'name': 'Kaila Deibler', 'scores': {'score': 15.89771199662455, 'type': 'exam'}}]}, {'_id': 81, 'fail': [{'name': 'Linnie Weigel', 'scores': {'score': 38.33608066073369, 'type': 'homework'}}]}, {'_id': 77, 'fail': [{'name': 'Efrain Claw', 'scores': {'score': 31.30359328252952, 'type': 'quiz'}}]}, {'_id': 183, 'fail': [{'name': 'Cody Strouth', 'scores': {'score': 32.99854612126559, 'type': 'exam'}}]}, {'_id': 6, 'fail': [{'name': 'Jenette Flanders', 'scores': {'score': 37.32285459166097, 'type': 'exam'}}, {'name': 'Jenette Flanders', 'scores': {'score': 28.32634976913737, 'type': 'quiz'}}]}, {'_id': 60, 'fail': [{'name': 'Dusti Lemmond', 'scores': {'score': 17.27725327681863, 'type': 'exam'}}]}, {'_id': 138, 'fail': [{'name': 'Jesusa Rickenbacker', 'scores': {'score': 15.6237624645333, 'type': 'exam'}}, {'name': 'Jesusa Rickenbacker', 'scores': {'score': 7.856092232737, 'type': 'quiz'}}]}, {'_id': 0, 'fail': [{'name': 'aimee Zank', 'scores': {'score': 1.4631797367

7)Create a new collection which consists of students who scored above pass mark in all the categories.


In [67]:
students7 = records.aggregate([
      { "$unwind": "$scores" },
      { "$match": { "scores.score": { "$gt": 40} }
       },
       { "$group": {"_id": "$_id", "pass": { "$push": { "name": "$name", "scores": "$scores"}}
       }
       },
       {"$out": "studentspassed"}
      ])

In [68]:
students7 = db.studentspassed.find()
print(list(students7))

[{'_id': 30, 'pass': [{'name': 'Kaila Deibler', 'scores': {'score': 56.93965183412178, 'type': 'quiz'}}, {'name': 'Kaila Deibler', 'scores': {'score': 66.64493295066322, 'type': 'homework'}}]}, {'_id': 81, 'pass': [{'name': 'Linnie Weigel', 'scores': {'score': 66.0349256424749, 'type': 'exam'}}, {'name': 'Linnie Weigel', 'scores': {'score': 67.57096025532985, 'type': 'quiz'}}]}, {'_id': 77, 'pass': [{'name': 'Efrain Claw', 'scores': {'score': 55.41266579085205, 'type': 'exam'}}, {'name': 'Efrain Claw', 'scores': {'score': 88.73134194093676, 'type': 'homework'}}]}, {'_id': 183, 'pass': [{'name': 'Cody Strouth', 'scores': {'score': 78.61720316992681, 'type': 'quiz'}}, {'name': 'Cody Strouth', 'scores': {'score': 89.62847560459466, 'type': 'homework'}}]}, {'_id': 188, 'pass': [{'name': 'Leola Lundin', 'scores': {'score': 60.314725741828, 'type': 'exam'}}, {'name': 'Leola Lundin', 'scores': {'score': 41.12327471818652, 'type': 'quiz'}}, {'name': 'Leola Lundin', 'scores': {'score': 74.86991